# Generate Dyanmical Regression Matrix


Import Libraries


In [1]:
import sympy as p
import numpy as np
import sys

# SYMBOLIC.py is in parent directory
sys.path.append("..")
from SYMBOLIC import *

## 1. Kinematics with PoE


### 1.1 Variable Definitions


Define time $t$ and angles.


In [2]:
t = p.Symbol("t")

# Differentialable Variables
theta_1_t = p.Function("theta_1")(t)
theta_2_t = p.Function("theta_2")(t)

# Undifferentialable Variables
theta_1, theta_2 = p.symbols("theta_1 theta_2")
dtheta_1, dtheta_2 = p.symbols("dtheta_1 dtheta_2")

Define robot parameters.


In [3]:
m1, m2, l1, l2, lc1, lc2, Jc1, Jc2, g = p.symbols("m1 m2 l1 l2 lc1 lc2 Jc1 Jc2 g")

Define substitution functions from differentialable variables to undifferentialable variables.

In [15]:
def ThetaT2Theta(Expression):
    return (
        Expression.subs(theta_1_t.diff(t), dtheta_1)
        .subs(theta_2_t.diff(t), dtheta_2)
        .subs(theta_1_t, theta_1)
        .subs(theta_2_t, theta_2)
    )


def Theta2ThetaT(Expression):
    return Expression.subs(theta_1, theta_1_t).subs(theta_2, theta_2_t)

Define geometric parameters. We apply **Modified D-H** notation. Noting that $\square_{i}$ refers to $\square$ of joint $i$, $\square_{ci}$ refers to $\square$ of rod $i$.


In [5]:
# Joint DH Parameters
x0, x1, x2 = p.symbols("x_0 x_1 x_2")
y0, y1, y2 = p.symbols("y_0 y_1 y_2")
z0, z1, z2 = p.symbols("z_0 z_1 z_2")

# Link CoM Parameters
xc1, xc2 = p.symbols("x_c1 x_c2")
yc1, yc2 = p.symbols("y_c1 y_c2")
zc1, zc2 = p.symbols("z_c1 z_c2")

From the configuration, we got

In [6]:
# Joint Parameters
x0, y0, z0 = 0, 0, 0
x1, y1, z1 = 0, 0, 0
x2, y2, z2 = l1, 0, 0

# CoM Parameters
xc1, yc1, zc1 = lc1, 0, 0
xc2, yc2, zc2 = lc2, 0, 0

# RPY Angles
RPY_0_1 = [0, 0, 0]
RPY_1_2 = [0, 0, 0]
RPY_1_c1 = [0, 0, 0]
RPY_2_c2 = [0, 0, 0]

### 1.2 Forward Kinematics


Transformation matrixes at $\vec{0}$. `T_i_j_0` refers to $T_{i}^{j} (\vec{0}) $.


In [7]:
T_1_0_0 = Translation_4x4(p.Matrix([x1, y1, z1])) @ Rotation_RPY_4x4(RPY_0_1)
T_2_1_0 = Translation_4x4(p.Matrix([x2, y2, z2])) @ Rotation_RPY_4x4(RPY_1_2)
T_c1_1_0 = Translation_4x4(p.Matrix([xc1, yc1, zc1])) @ Rotation_RPY_4x4(RPY_1_c1)
T_c2_2_0 = Translation_4x4(p.Matrix([xc2, yc2, zc2])) @ Rotation_RPY_4x4(RPY_2_c2)

Joint positions at $\vec{0}$.

In [8]:
def Position(Transformation_i_j, P_i=p.Matrix([0, 0, 0, 1])):
    return (Transformation_i_j @ P_i)[:3, :]


T_1_0_0 = T_1_0_0
T_2_0_0 = T_1_0_0 @ T_2_1_0
T_c1_0_0 = T_1_0_0 @ T_c1_1_0
T_c2_0_0 = T_2_0_0 @ T_c2_2_0

r_1_0 = Position(T_1_0_0)
r_2_0 = Position(T_2_0_0)
r_c1_0 = Position(T_c1_0_0)
r_c2_0 = Position(T_c2_0_0)

Joint poses at $\vec{0}$.

In [9]:
omega_1_0 = p.Matrix([0, 0, 1]).reshape(3, 1)
omega_2_0 = Rotation_RPY_4x4(RPY_1_2)[:3, :3] @ omega_1_0

Joint unitary twists at $\vec{0}$.

In [10]:
xi_1_0 = Joint2Twist(omega_1_0, r_1_0)
xi_2_0 = Joint2Twist(omega_2_0, r_2_0)

Transformation matrixes at $\vec{\theta}$ computed with PoE.


In [11]:
T_1_0 = MatrixExp_4x4(xi_1_0, theta_1_t) @ T_1_0_0
T_2_0 = MatrixExp_4x4(xi_1_0, theta_1_t) @ MatrixExp_4x4(xi_2_0, theta_2_t) @ T_2_0_0
T_c1_0 = MatrixExp_4x4(xi_1_0, theta_1_t) @ T_c1_0_0
T_c2_0 = MatrixExp_4x4(xi_1_0, theta_1_t) @ MatrixExp_4x4(xi_2_0, theta_2_t) @ T_c2_0_0

Joint positions and poses at $\vec{\theta}$.

In [21]:
# Positions
r_1 = Position(T_1_0)
r_2 = Position(T_2_0)

# Poses
omega_1 = p.simplify(MatrixExp_3x3(omega_1_0, theta_1_t) @ omega_1_0)
omega_2 = p.simplify(
    MatrixExp_3x3(omega_1_0, theta_1_t)
    @ MatrixExp_3x3(omega_2_0, theta_2_t)
    @ omega_2_0
)

Joint velocities at $\vec{\theta}$.


In [17]:
v_1 = ThetaT2Theta(p.simplify(r_1).diff(t))
v_2 = ThetaT2Theta(p.simplify(r_2).diff(t))

## 2. Dynamical Regression Matrix


In [53]:
def Ttilde_10x1(OmegaVec, VVec, TransformationMat):
    return ThetaT2Theta(
        (1 / 2 * OmegaVec.T @ KMatrix_3x6(OmegaVec))
        .row_join(VVec.T @ Vector2Matrix_3x3(OmegaVec) @ TransformationMat[:3, :3])
        .row_join(1 / 2 * VVec.T @ VVec)
    ).T


p.simplify(Ttilde_10x1(omega_2, v_2, T_2_0)).T

Matrix([[0, 0, 0, 0, 0, 0.5, dtheta_1*l1*cos(theta_2), -dtheta_1*l1*sin(theta_2), 0, 0.5*dtheta_1**2*l1**2]])

In [54]:
def Vtilde_10x1(rVec, gVec, TransformationMat):
    return ThetaT2Theta(
        p.Matrix([[0, 0, 0, 0, 0, 0]])
        .row_join(-gVec.T @ TransformationMat[:3, :3])
        .row_join(-gVec.T @ rVec)
    ).T


Vtilde_10x1(r_1, p.Matrix([[0, 0, -g]]).T, T_1_0).T

Matrix([[0, 0, 0, 0, 0, 0, 0, 0, g, 0]])